# FARA
> Play FARA

- https://playfara.com

Motivation: A large part of playing FARA involves going through certain routines. These routines are fun to figure out, but their novelty wears off as the game progresses. When starting a new game of FARA, it is preferable to thoroughly search, harvest, and craft in the initial locality. It would be nice to automate away this early game routine. Another such routine is preparing for specific journeys which could involve harvesting and preparing shelter, meals, and rest.

- [ ] **Available Moves**: Learn what the available moves are by parsing the gamelog. And maybe in which situations they are useful.
- [ ] **Satchel**: Learn to craft a burlap satchel in the first area.

Other ideas that could be fun:

- draw a map from the gamelog
- recuperate

Boring ideas that could be done:

- farming :-1:

## Available Moves

- Given a game log ending with `> `, output the next command.
- Probably want to start with a manual played game log. This would bootstrap the available commands. The game should be played without a map, so that we 
- Labeling. As long as there is a valid, non-empty result, I think we can assume that the label is positive. Even if it leads to death. 
- We might need to know how to determine that the game ended, etc.. But we probably don't need to do anything too sophisticated. We could just cap the log at several thousand attempts to take a turn.
- Remember, that we're going to be generating the game logs in one phase, but training in a separate phase. Even when we get around to self play, it should be separated.

## Satchel

- The actual recipe is fairly constant. But it may be worth it to learn the recipe through a model. There is some need to understand English in order to discover the ingredients. For example, understanding "2 Rolls of Burlap" vs. "Roll of Burlap".
- Could a bot say which ingredients are necessary given a target item?
- The initial map and its resource location is random.

If coding the algorithm, it'd be something like this:
1. Identify ingredients to gather by recursively calling `/craft`.
2. Find and Harvest the ingredients.
3. Craft the ingredients.

In [20]:
move("/look", debug=True)

total 104
drwxr-xr-x  3 ewolfson  wheel     96 Feb 28 18:28 ..
-rw-r--r--@ 1 ewolfson  wheel    414 Mar  1 15:24 The Life and Times of bot the Soldier(1).txt
-rw-r--r--@ 1 ewolfson  wheel  20898 Mar  1 16:09 The Life and Times of bot the Soldier(2).txt
-rw-r--r--@ 1 ewolfson  wheel  21536 Mar  1 16:09 log
drwxr-xr-x  5 ewolfson  wheel    160 Mar  1 16:09 .


## Starting FARA 

Initialize the Selenium driver.
Unless you use headless mode, you'll want to keep this new Firefox window open as long as you are playing.

At this point, we will go through the steps necessary to start a game. And do so in a way that the game log can be saved. We'll use the log later to generate training data.

Time to get it saving somewhere more manageable. I'll try to overwrite the same file for now though.

In [2]:
! rm -f /tmp/fara/game/*
! ls /tmp/fara/game/


In [3]:
import os

from selenium import webdriver

# TODO: patch to a class
def set_logging_dir(p):
  log_dir = p
  ! mkdir -p $p
  ! rm $p/*
  fp = webdriver.FirefoxProfile()

  fp.set_preference("browser.download.folderList",2)
  fp.set_preference("browser.download.manager.showWhenStarting",False)
  fp.set_preference("browser.download.dir", p)
  # FIXME: these aren't working.
  # fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/plain")
  fp.set_preference("browser.helperApps.alwaysAsk.force", False)

  return (fp, p)

In [192]:
from selenium.webdriver.firefox.options import Options
import time

def start_game():
  driver = webdriver.Firefox()
  try: driver.quit()
  except: pass
  options = Options()
  # options.headless = True
  fp, log_dir = set_logging_dir("/tmp/fara/game")
  driver = webdriver.Firefox(options=options,
    firefox_profile=fp
  )
  time.sleep(1)
  driver.get("https://brianiscreative.itch.io/fara")
  time.sleep(1)
  # TODO: Wait until it's loaded
  from selenium.webdriver.common.by import By
  driver.find_element_by_class_name("load_iframe_btn").click()
  time.sleep(1)
  # TODO: Wait? until it's loaded
  g = driver.find_element(by="tag name", value="body")
  [g.send_keys(c+"\n") for c in ["/setname bot", "/setclass soldier"]]
  time.sleep(1)
  [g.send_keys(c+"\n") for c in ["/1", "/setdesc", "/stopsharing", "/hd", "/showinput", "/logging"]]
  
  # Since the neverAsk option isn't working, we'll need to force the save via the `/savelog` command.
  # The user will need to click "Save File" and "Do this automatically for files like this from now on."
  g.send_keys("/savelog\n")
  input("""
    Since the neverAsk option isn't working, we'll need to force the save via the `/savelog` command.
    The user will need to click "Save File" and "Do this automatically for files like this from now on."
    Hit enter in this prompt when this is done.
  """)
  
  return g, log_dir


In [6]:
%%time
g, log_dir = start_game()

rm: /tmp/fara/game/*: No such file or directory
CPU times: user 33.4 ms, sys: 24.5 ms, total: 57.9 ms
Wall time: 14.5 s


The log is now ready to be saved. 

If it was set up by the expectations above, the log should be output to a file that should starts with `The Life and Times of bot the Soldier` and ends with `.txt`.

For now the gamelog is cumulative, but it will be possible [to clear it at some point](https://discord.com/channels/448497182392451073/448498300723789845/814283733988933652). 

Also at some point, we can try to autosave the file with this FAQ: https://selenium-python.readthedocs.io/faq.html?highlight=file#how-to-auto-save-files-using-custom-firefox-profile

In [9]:
!cat /tmp/fara/game/*

Gaming logging  ENABLED
> /savelog


## Making Moves

Let's set up a short function to make a move and output the game log after each move.

In [21]:
def move(m, debug=False):
  [g.send_keys(x + "\n") for x in [m, "/savelog"]]
  ! mv '{log_dir + "/The Life and Times of bot the Soldier.txt"}' '{log_dir + "/log"}'
  if debug:
    ! ls -latr {log_dir}
    ! tail {log_dir}/log
  

In [11]:
! echo $log_dir

/tmp/fara/game


In [36]:
move("/lt", debug=True)

total 160
drwxr-xr-x  3 ewolfson  wheel     96 Feb 28 18:28 ..
-rw-r--r--@ 1 ewolfson  wheel    414 Mar  1 15:24 The Life and Times of bot the Soldier(1).txt
-rw-r--r--@ 1 ewolfson  wheel  20898 Mar  1 16:09 The Life and Times of bot the Soldier(2).txt
-rw-r--r--@ 1 ewolfson  wheel  22604 Mar  1 16:15 The Life and Times of bot the Soldier(3).txt
-rw-r--r--@ 1 ewolfson  wheel  26060 Mar  1 16:22 log
drwxr-xr-x  6 ewolfson  wheel    192 Mar  1 16:22 .
> /savelog
> /listexits
 There is an exit 8 steps to the southeast. 
> /savelog
> /savemap
'/savemap' is not a valid command.
> /savelog
> /lt
You see 3 Limes  five steps to the southeast,  some Alder Tree Seeds  five steps to the south,  some Alder Tree Seeds  two steps to the northwest,  a Young Dog  five steps to the south,  a Campfire you are targeting right next to you,  a Tin Shortsword within reach to the southeast,  a Wooden Log  two steps to the northwest,  a Wooden Log  five steps to the south,  a Stick  five steps to the south,  

There are a lot of moves that can be made.
We are interested in training the bot to work in exploration mode. Not the travel mode.

Let's list a few available moves.

In [18]:
MOVES = ["/" + item for sublist in
          [
            ["move " + d for ds in ["n ne e se s sw w nw".split(" ")] for d in ds],
            ["look", "use", "recipes"],
            ["craft " + x for x in ["burlap satchel"]]
          ] for item in sublist
        ]
MOVES

['/move n',
 '/move ne',
 '/move e',
 '/move se',
 '/move s',
 '/move sw',
 '/move w',
 '/move nw',
 '/look',
 '/use',
 '/recipes',
 '/craft burlap satchel']

That should make for a nice format to store available moves in for debugging. Might not need it though.

Let's move on to parsing the game log.

## The Gamelog

### Persisting it

First we'll want to copy it over somewhere more persistent than the temp directory.

In [168]:
def persist_log(src=None, tgt=None):
  if not src:
    src = f"{log_dir}/log"
  if not tgt:
    ts = ! date +%s
    ts=ts[0]
    tgt = f"fara_logs/{ts}"
  !cp {src} {tgt}
  print(tgt)

In [169]:
# ! rm fara_logs/*

In [170]:
ts = ! date +%s
ts=ts[0]
persist_log(f"{log_dir}/log", f"fara_logs/{ts}")
!tail fara_logs/{ts}

fara_logs/1614638345
> /savelog
> /listexits
 There is an exit 8 steps to the southeast. 
> /savelog
> /savemap
'/savemap' is not a valid command.
> /savelog
> /lt
You see 3 Limes  five steps to the southeast,  some Alder Tree Seeds  five steps to the south,  some Alder Tree Seeds  two steps to the northwest,  a Young Dog  five steps to the south,  a Campfire you are targeting right next to you,  a Tin Shortsword within reach to the southeast,  a Wooden Log  two steps to the northwest,  a Wooden Log  five steps to the south,  a Stick  five steps to the south,  2 Sticks  five steps to the southeast,  a Wooden Log  five steps to the east,  and  a Stick  five steps to the east.
> /savelog


Now that it's persisted, let's extract all the commands from the saved log. 

### Parsing it

Let's not lose track of the goal here:

> Given a game log ending with `> `, output the next command.

I think this parsing is just a way to build up a custom vocabulary.

In [147]:
def extract_inputs(p=None):
  if not p:
    p = !ls fara_logs/ | tail -n 1
    p = f"fara_logs/{p[0]}"
  lines = !cat {p}
  inputs = [line[2:] for i, line in enumerate(lines)
            if line[0:2] == "> " and line not in ["> " + bl for bl in ["/savelog", "/hd", "/help"]]
            and (not lines[i+1].endswith("is not a valid command."))
           ]
  return inputs

In [143]:
# !grep 'valid' fara_logs/`ls fara_logs/ | tail -n 1`

In [160]:
print(set(extract_inputs()))

{'/look', '/targets', '/look 3', '/wear burlap satchel', '/approach 2', '/showinput', '/take 1', '/lt', '/listitems', '/approach alder tree to the north', '/inspect Adventurer', '/inspect alder tree', '/look 4', '/approach 5', '/inspect 4', '/harvest 1', '/craft burlap satchel', '/inventory', '/logging', '/harvest 5', '/listexits', '/craft roll of burlap', '/look at Alder Tree', '/approach 10', '/craft plant fiber', '/lh', '/approach 7', '/approach alder tree', '/craft 2 rolls of burlap', '/lw', '/harvest 2', '/status', '/craft plant fibers', '/inspect 10', '/li', '/craft rolls of burlap', '/look Alder Tree', '/inspect 3'}


Next we could try to build a labeled data set. But we should probably hit the breaks, step back, and build a random bot first. Once we have something that can make valid but random moves, we can try to substitute the random bot for a trained both. We should be able to re-use the skeleton from the random bot.

## Random Strategy

Soon we can move on to predicting the next move. But for now, let's create something that can generated a game log from a hard-coded list of inputs.

To start, our inputs can come from our latest game log.

In [198]:
import random

def game(strategy='random'):
  moves = list(set(extract_inputs()))
  if not moves:
    moves = MOVES
  
  g, log_dir = start_game()
  STOP = 3
  for t in range(STOP):
    if strategy == 'random':
      m = random.choice(moves)
      print(m)
      move(m, debug=False)
  persist_log()
  return moves, g, log_dir

In [199]:
game()
# g.quit()


    Since the neverAsk option isn't working, we'll need to force the save via the `/savelog` command.
    The user will need to click "Save File" and "Do this automatically for files like this from now on."
    Hit enter in this prompt when this is done.
   


/lt


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59790): Max retries exceeded with url: /session/daf04d97-dc64-7f4d-acb1-62289c53b9c5/element/961b2ed4-002f-e04c-898d-8a9bab8909cc/value (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x108df44f0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [197]:
! cat fara_logs/1614638985

Gaming logging  ENABLED
> /savelog
> /lt
You see a Shrouded Figure you are targeting  nine steps to the northwest,  a Tin Shortsword  eight steps to the south,  a Campfire  nine steps to the south,  and  an Adventurer  nine steps to the northwest.
> /savelog
> /inspect 10
You inspect a Shrouded Figure about nine steps to the northwest. A glowing, viscous liquid oozes from this ghostly apparition's eyes and mouth.  It is  holding an :{Adamantine Scythe}:. It is Hostile. It is imbued with arcane energy.  It is about your size.  It is  8 steps out of your attack range.  Engaging it in combat would surely be your undoing.  
...An Adventurer is bleeding heavily! 
...An Adventurer dies by bleeding. 
...An Adventurer drops a Wooden Breastplate, a Tattered Note,  and a Key Fragment. 
> /savelog
> /lw
You are not wearing anything.
> /savelog


## Interface via Selenium

:warning: this is just cruft that came from `cyber-wanderer` branch. Could be useful at some point.

In [ ]:
from selenium.webdriver.common.keys import Keys
import random
import re
import io
import sys

lm = [
    Keys.ARROW_DOWN, Keys.ARROW_LEFT, Keys.ARROW_UP, Keys.ARROW_RIGHT,
    '.', 'x'
]

#################
# Random Strategy
# effectively, causes staff to be used less often
# longer games, but not necessarily more points.
costs = [1/cost for cost in [1,1,1,1,1,10]]
def random_move(b):
    return random.choices(lm, costs)
#################

#################
# Manual Strategy [WIP]
def human_move(b):
    return sys.stdin.read(1)
#################

def board(g):
    # .text is a little too magical (writes \n instead of <br>, etc..)
    return g.find_element(by="id", value="tsv").get_attribute('innerHTML')

def game_score(g):
    return None
#     # .text is easier to regex than innerHTML
#     game_over = re.search("Your final score is (\d+) after ", g.text)
#     # scoring function is the score, but alternative scoring
#     # functions could count kills, visits, keys, etc..
#     return game_over.group(1) if game_over else None

def turn(g):
    s = game_score(g)
    b = board(g)
    # How we get m will by the strategy
    m = random_move(b)
    # m = human_move(board(g))
    if not s: g.send_keys(m)
    return (s, f"{b}\t{m}")

def turn_score(b, g):
    return 0
#     try:
#         return int(re.match("Self: (?:\<[^>]+\>)*(\w+)", b.split("\t")[2500]).group(1))
#     except ValueError:
#         return int(re.search("Your final score is (\d+) after ", g.text).group(1))

def reward(b1, b2, g):
    return turn_score(b2, g) - turn_score(b1, g)

# def game(driver, save_as=None):
#     driver.get("https://brianiscreative.itch.io/fara")
#     g = driver.find_element(by="tag name", value="body")
#     [g.send_keys(c) for c in ["/stopsharing", "/hd", "/showinput", "/logging"]]
#     score = None
#     history = []
#     while score is None:
#         score, b = turn(g)
#         r = reward(b, board(g), g)
#         # make sure not to leak the moves and rewards in training
#         history.append(f"{b}\t{r}")
#     # print(history)
#     if save_as:
#         with open(f"{save_as}", "w") as f: f.write("\n".join(history))
#     return int(score), len(history)